In [25]:
import numpy as np
import matplotlib.pyplot as plt

In [168]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [169]:
X

array([[1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00,
        1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00],
       [1.00e+00, 1.00e+00, 2.00e+00, 1.00e+00, 3.00e+00, 0.00e+00,
        5.00e+00, 1.00e+01, 1.00e+00, 2.00e+00],
       [5.00e+02, 7.00e+02, 7.50e+02, 6.00e+02, 1.45e+03, 8.00e+02,
        1.50e+03, 2.00e+03, 4.50e+02, 1.00e+03],
       [1.00e+00, 1.00e+00, 2.00e+00, 1.00e+00, 2.00e+00, 1.00e+00,
        3.00e+00, 3.00e+00, 1.00e+00, 2.00e+00]])

In [170]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

In [171]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [172]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

In [173]:
X_st

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  3.        ,
         0.        ,  5.        , 10.        ,  1.        ,  2.        ],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  2.        ,
         1.        ,  3.        ,  3.        ,  1.        ,  2.        ]])

In [174]:
X_st.shape

(4, 10)

In [177]:
def calc_logloss(y, y_pred):
    i_zero = np.where(y_pred == 0)
    i_one = np.where(y_pred == 1)
    
    for i in i_zero:
        y_pred[i] = 1e-5
    for i in i_one:
        y_pred[i] -= 1e-5

    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [178]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [179]:
### Logistic Regression

In [180]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
    if i % (iterations / 10) == 0:
        print(i, W, err)
    return W

In [273]:
W = eval_model(X_st, y, iterations=10000, alpha=1e-2)

10000 [-2.77079473 -0.99580928  0.56650766  3.2676589 ] 0.4058305387773311


Задание 3: Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 
    (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [274]:
def calc_pred_proba(W, X):
    y_pred_proba = 1 / (1 + np.exp(-np.dot(W, X)))
    return y_pred_proba

In [275]:
y_pred_proba = calc_pred_proba(W, X)
y_pred_proba

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [276]:
def calc_pred(W, X):
    y_pred_proba = calc_pred_proba(W, X)
    y_pred = np.zeros(y_pred_proba.shape[0])
    
    for i in range(len(y_pred_proba)):
        if i >= 0.5:
            y_pred[i] = 1
    return y_pred

In [277]:
y_pred = calc_pred(W, X)
y_pred

array([0., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

Задание 5: Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [280]:
def accuracy(y, y_pred):
    sum = 0
    for i in range(y.shape[0]):
        if y[i] == y_pred[i]:
            sum += 1
    return sum / y.shape[0]
    

In [281]:
accuracy = accuracy(y, y_pred)
accuracy

0.6

In [299]:
def err_matrix(y, y_pred):
    zero_matrix = np.zeros((2, 2))
    TP, FP, FN, TN = 0, 0, 0, 0
    for i in range(y.shape[0]):
        if y[i] == 1 and y_pred[i] == 1:
            TP += 1
        if y[i] == 0 and y_pred[i] == 1:
            FP += 1
        if y[i] == 1 and y_pred[i] == 0:
            FN += 1
        if y[i] == 0 and y_pred[i] == 0:
            TN += 1
    
    zero_matrix[0][0] = TP
    zero_matrix[0][1] = FP
    zero_matrix[1][0] = FN
    zero_matrix[1][1] = TN
    return zero_matrix
       

In [300]:
err_matrix= err_matrix(y, y_pred)
err_matrix

array([[5., 4.],
       [0., 1.]])

In [341]:
def precision(err_matrix):
    return err_matrix[0][0] / (err_matrix[0][0] + err_matrix[0][1])    

In [342]:
def recall(err_matrix):
    return err_matrix[0][0] / (err_matrix[0][0] + err_matrix[1][0])    

In [343]:
precision = precision(err_matrix)
recall = recall(err_matrix)
print(f'presion: {precision}, recall: {recall}')

presion: 0.5555555555555556, recall: 1.0
